# Example: Random Feature ROM with NREL Wind Data

RF = Random Feature 

In [1]:
import rom
import numpy as np

The NACA 0012 drag data can be found [here](https://github.com/kaylabollinger/ROM/tree/main/data/NACAdrag). To load the data, just replace the data_dir variable with the appropriate path to the dataset.

In [2]:
data_dir = '../data/NRELwind/NRELwind_RootMxc1'
X = np.load(data_dir+'/X.npy')
Y = np.load(data_dir+'/Y.npy').reshape(-1)

Designate the number of (randomly chosen) data points to train with.

In [3]:
num_train = 10000

In [4]:
ind = list(range(X.shape[0]))
np.random.shuffle(ind)
ind_train = ind[:num_train]

In [5]:
X_train = X[ind_train,:]
Y_train = Y[ind_train]

Since derivative information is not available for this dataset, but we would like to use active subspaces for the dimension reductions step, we need to approximate derivatives. We use local linear gradients with $n=60$ nearest neighbors.

In [6]:
gr = rom.gradients
DY_train = gr.local_linear_gradients(X_train,Y_train,n=60)

C:\Users\knb_w\anaconda3\lib\site-packages\rom-0.1.dev93+g6280f1f-py3.9.egg\rom\gradients.py:38: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


To train the RF model, we first learn the $k$ dimensional linear subspace $U$ via active subspaces--see [Bollinger (2022)](???) (link will be available once published) for more detail. If given datapoint is labeled $x$, then reduced data is given by $U^T x$, labeled "UTX" in this code.

In [7]:
k = 1

ss = rom.subspaces
U = ss.AS(DY_train,k)

UTX = np.matmul(X,U)
UTX_train = np.matmul(X_train,U)

Then, we train the RF surrogate model. For this example, our hyperparameter choice will be:
- N = number of random features = num_train/2
- alpha = regularizing parameters = 1e-3
- s = sparsity order on random weights = k

In [8]:
model = rom.surr_model.RF()
model.train([UTX_train, Y_train],N=int(np.ceil(num_train/2)),alpha=1e-3,s=k)

Using the trained model, we predict output over the entire dataset and report error.

In [9]:
Y_calc = model.predict(UTX)

print(f'relative error = {rom.utils.rel_error(Y,Y_calc)}')

relative error = 0.005157119148963896
